### Задача (варіант 1)

**Ієрархія:** p = 3, повна, m = (4, 3, 3)

<img src="images/lab_2_hierarchy.png" width=256>

**Метод розрахунку глобальних ваг:** дистрибутивний, ГВБВПА

*Методи аналізу альтернатив рішень на основі ієрархічної моделі критеріїв (МАІ, analytic hierarchy process, AHP)* складаються з наступних чотирьох загальних етапів:

1. Побудова ієрархічної моделі критеріїв, цілей та інших факторів, які впливають на головну ціль прийняття рішення; побудова множини альтернативних варіантів рішень.

2. Отримання суджень експертів щодо парних порівнянь елементів одного рівня ієрархії відносно спільного елементу вищого рівня. Парні порівняння проводяться у вибраній шкалі і за результатами будуються матриці парних порівнянь (МПП), які є обернено симетричними.

3. Математична обробка суджень експертів:
   * розрахунок локальних ваг елементів кожного рівня ієрархії відповідно до батьківських елементів вищого рівня на основі МПП; побудова локальних ранжувань;
   * аналіз узгодженості експертних оцінок;
   * розрахунок глобальних ваг елементів ієрархії відносно головної цілі прийняття рішення, використовуючи методи агрегування; побудова ранжування на основі глобальних ваг.

4. Аналіз чутливості отриманих ранжувань.

Постановка задачі:

**Дано:** 
 * $A = \{A_i | i = 1, \dots, N \}$ - множина альтернативних варіантів рішень;
 * $C = \{C_j | j = 1, \dots, M \}$ - множина критеріїв оцінювання альтернатив;
 * $a_{ij}$ - ненормована вага льтернативи $A_i$ за критерієм $C_j$;
 * $w_j^C$ - вага критерію $C_j, \sum_{j=1}^{M} w_j^C = 1$

**Потрібно:**
 * знайти глобальні ваги $w_i^{глоб}$ альтернатив $A_i, i = 1, \dots, N$

***Методи аналізу альтернатив рішень на основі ієрархічної моделі критеріїв (МАІ, analytic hierarchy process, AHP) складаються з наступних чотирьох загальних етапів:***


***1.*** Побудова ієрархічної моделі критеріїв, цілей та інших факторів, які впливають на головну ціль прийняття рішення; побудова множини альтернативних варіантів рішень - цей етап зроблено за нас - ієрархія (4, 3, 3) (тобто (1, 4, 3, 3) враховуючи мету)

In [1]:
import numpy as np
# put some fixed seed for reproducibility of the results
np.random.seed(42)

In [2]:
# we work with fully connected hierarcy, its architecture is defined by number of nodes per layer
# All layers but the last 'layers[:-1]' - creiterions
layers = [1, 4, 3, 3]

Функція для генерування коректної матриці парних порівнянь для мультиплікативної групи

In [3]:
def generate_matched_matrix(dim, low=1, high=10):
    """
    Generates matched multiplicational matrix mmp i.e. \forall i \in [0, n-1]: mpp[i][i] = 1;
    \forall i, j i \neq j: mpp[i][j] = 1 / mpp[j][i]
    Returns 2D matched matrix of shape (dim, dim)
    """
    mmp = np.zeros((dim, dim))
    temp = np.random.randint(low, high, size=dim).astype(float)
    powers = np.random.choice(3, size=dim, p=[0.5, 0, 0.5]) - 1
    mmp[0, :] = np.power(temp, powers)
    j = 0
    for i in range(1, dim):
        mmp[i, j] = 1 / mmp[j, i]
    for i in range(1, dim):
        for j in range(1, dim):
            mmp[i, j] = mmp[i, 0] * mmp[0, j]
    mmp[0][0] = 1
    return mmp

# Data generation

***2.*** Отримання суджень експертів щодо парних порівнянь елементів одного рівня ієрархії відносно спільного елементу вищого рівня. Парні порівняння проводяться у вибраній шкалі і за результатами будуються матриці парних порівнянь (МПП), які є обернено симетричними.

Генеруємо матриці парних порівнянь - замість опитування експертів.

Наша ієрархія - (4, 3, 3), фактично працюємо з ієрархією (1, 4, 3, 3). Тобто експертами формуються наступні матриці парних порівнянь
* 4x4x1 - парні порівняння 4 критеріїв відносно 1 критерію (мети)
* 3x3x4 - парні порівняння 3 підкритеріїв відносно 4 критеріїв
* 3х3х3 - парні порівняння 3 альтернатив відносно 3 підкритеріїв

Отримані дані об'єднаємо в список.

In [4]:
# generally speaking, number of nodes can vary from layer to layer, so further we use nested lists,
# not a np.array

D = []
for idx_curr in range(1, len(layers)):
    d = np.empty((layers[idx_curr], layers[idx_curr], layers[idx_curr - 1]))
    for i in range(layers[idx_curr - 1]):
        mpp = generate_matched_matrix(layers[idx_curr])
        d[:, :, i] = mpp
    D.append(d)

# display shapes and values of generated MPPs
def print_D(D):
    for mpp in D:
        print(mpp.shape)
        print(mpp)
        print("*" * 80)
print_D(D)

(4, 4, 1)
[[[1.00e+00]
  [2.50e-01]
  [1.25e-01]
  [5.00e+00]]

 [[4.00e+00]
  [1.00e+00]
  [5.00e-01]
  [2.00e+01]]

 [[8.00e+00]
  [2.00e+00]
  [1.00e+00]
  [4.00e+01]]

 [[2.00e-01]
  [5.00e-02]
  [2.50e-02]
  [1.00e+00]]]
********************************************************************************
(3, 3, 4)
[[[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
  [1.25000000e-01 2.00000000e+00 1.00000000e+00 3.33333333e-01]
  [8.00000000e+00 2.00000000e-01 3.33333333e-01 7.00000000e+00]]

 [[8.00000000e+00 5.00000000e-01 1.00000000e+00 3.00000000e+00]
  [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
  [6.40000000e+01 1.00000000e-01 3.33333333e-01 2.10000000e+01]]

 [[1.25000000e-01 5.00000000e+00 3.00000000e+00 1.42857143e-01]
  [1.56250000e-02 1.00000000e+01 3.00000000e+00 4.76190476e-02]
  [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]]]
********************************************************************************
(3, 3, 3)
[[[1.00000

In [5]:
# generate weight of each criteria on the first layer
# on further layers we recompute 'weights' of elements of new layer
criteria_weights = np.random.rand(layers[1])
criteria_weights /= np.linalg.norm(criteria_weights, ord=1)
print(f"Criteria weights in given hierarchy: \n{criteria_weights}")

Criteria weights in given hierarchy: 
[0.28010459 0.26678609 0.17825941 0.27484991]


***3.*** Математична обробка суджень експертів:

* розрахунок локальних ваг елементів кожного рівня ієрархії відповідно до батьківських елементів вищого рівня на основі МПП; побудова локальних ранжувань;
* аналіз узгодженості експертних оцінок;
* розрахунок глобальних ваг елементів ієрархії відносно головної цілі прийняття рішення, використовуючи методи агрегування; побудова ранжування на основі глобальних ваг.

Обраховуємо локальні ваги використовуючи підхід EM з попередньої лабораторної роботи.

**ЕМ метод** Згідно цього метода вектором ваг є власний вектор МПП, що відповідає її найбільшому власному числу. Тобто кожна МПП буде замінена вектором тієї ж розмірності, що і МПП. Отже на виході функції мусимо отримати список тензорів наступної розмірності:

* 4х1
* 3х1
* 3х1

In [6]:
def _get_main_w_v_power(mat):
    """
    Computes main eigenvalue and main eigenvector.
    return: main eigenvalue and eigenvector
    """
    x = np.ones(mat.shape[0])
    x /= np.linalg.norm(x, ord=1)
    k = 30
    D = mat
    for idx in range(k):
        l = np.linalg.norm(np.dot(D, x), ord=1)
        x = np.dot(D, x) / l
    return l, x

def calculate_local(D):
    """
    D: list of rank 3 tensors - MPPs - of shape (n_curr, n_curr, n_prev)
    return: list of rank 2 tensors - local weights w.r.t. criteria
    """
    # create list which we will return
    res = []
    for i in range(len(D)):
        res.append(np.empty(D[i].shape[1:]))
    # for each set of MPP for current layer
    for idx_mpps_group in range(len(D)):
        # we iterate over the last, 2nd dimension
        for idx_mpp in range(D[idx_mpps_group].shape[-1]):
            mpp = D[idx_mpps_group][:, :, idx_mpp]
            _, v_max = _get_main_w_v_power(mpp)
            res[idx_mpps_group][:, idx_mpp] = v_max
    return res
# local weights
v = calculate_local(D)
v

[array([[0.07575758],
        [0.3030303 ],
        [0.60606061],
        [0.01515152]]),
 array([[0.10958904, 0.15384615, 0.2       , 0.24137931],
        [0.87671233, 0.07692308, 0.2       , 0.72413793],
        [0.01369863, 0.76923077, 0.6       , 0.03448276]]),
 array([[0.09803922, 0.42857143, 0.70588235],
        [0.88235294, 0.14285714, 0.11764706],
        [0.01960784, 0.42857143, 0.17647059]])]

In [7]:
# check if we received normed local weights - compute sums of columns
for local in v:
    print(local.shape)
    for idx in range(local.shape[-1]):
        print(f"{idx} -> sum == {np.sum(local[:, idx])}")

(4, 1)
0 -> sum == 1.0
(3, 4)
0 -> sum == 1.0
1 -> sum == 0.9999999999999999
2 -> sum == 1.0
3 -> sum == 1.0
(3, 3)
0 -> sum == 1.0
1 -> sum == 1.0
2 -> sum == 1.0


Отже наразі отримали локальні ваги $a_{ij}$ критеріїв/альтернатив(на останньому рівні) відносно батьківських елементів вищого рівня.

# Methods for finding global normed weights of alternatives

## Methods for 2-level hierachies

### Метод дистрибутивного синтезу для ієрархії з 2 рівнів

In [8]:
# distributive synthesis approach to hierarchy with 2 levels
def distributive_synthesis_step(w_crit, v):
    """
    Computes global weights of alternatives in 2-level hierarchy
    w_crit: List of int
        weights of criteria
    v: ndarray, 2d matrix <num_of_alternatives> * <num_of_criterions>
        local weights of each alternative w.t.t. each criterion
    """
    n, m = v.shape
    # we need global weight for the each alternative
    w_glob = np.empty(n)
    # temporary matrix `r` with normed local weights
    r = np.empty(v.shape)
    for j in range(m):
        r[:, j] = v[:, j] / np.linalg.norm(v[:, j], ord=1)
    # calculate global weights, by definition - simplest possible code
    for i in range(n):
        acc = 0
        for j in range(m):
            acc += w_crit[j] * r[i, j]
        w_glob[i] = acc
    # norm output
    w_glob /= np.linalg.norm(w_glob, ord=1)
    return w_glob

<img src="images/lab_2_distributed_synthesis_example.png" width=650>

In [9]:
# test this method with data from lecturers example
w_crit_test = np.array([0.6, 0.4])
v_test = np.array([[2.52, 0.38],
              [0.31, 2.29],
              [1.26, 1.15]])
w_glob_correct = np.array([0.41, 0.29, 0.30])
w_glob = distributive_synthesis_step(w_crit_test, v_test)
np.testing.assert_array_almost_equal(w_glob, w_glob_correct, decimal=2)
print(f"Arrays are equal comparing to 2nd decimal: \nActual:       {w_glob} \nGround truth: {w_glob_correct}")

Arrays are equal comparing to 2nd decimal: 
Actual:       [0.40947273 0.28526735 0.30525992] 
Ground truth: [0.41 0.29 0.3 ]


### Метод ГВБВПА для ієрархії з 2 рівнів

In [10]:
# binary consideration of the binary preferences of alternatives
def binary_preferences_consideration_step(w_crit, v, output_P=False):
    """
    Computes global weights of alternatives in 2-level hierarchy.
    w_crit: List of int
        weights of criteria
    v: ndarray, 2d matrix <num_of_alternatives> * <num_of_criterions>
        local weights of each alternative w.t.t. each criterion
    output_P: boolean
        flag if it is necessary to print matrix of pair comparisons P (for debugging purposes)
    """
    # n - number of alternative, m - number of criterions
    n, m = v.shape
    # we want to fill in the next matrix of global weights pair comparison
    P = np.eye(n)
    # iterate over all pairs of different elements
    for i in range(1, n):
        for j in range(0, i):
            # preparation for utilization of `distributive_synthesis_step`
            v_tmp = v[(i, j), :]
            # utilization of `distributive_synthesis_step`
            w_glob_tmp = distributive_synthesis_step(w_crit, v_tmp)
            P[i, j] = w_glob_tmp[0] / w_glob_tmp[1]    # $\frac{w_{i}^{ik}}{w_{k}^{ik}}$
            P[j, i] = w_glob_tmp[1] / w_glob_tmp[0]
    if output_P:
        print(P)
    # reshaping is necessary for processing by `calculate_local` method
    P = [P.reshape((n, n, 1))]
    # so we have built 'matrix of pair comparisons'
    # we can treat it like the matrix of local weights and use, for example, EM method to compute weights
    res = calculate_local(P)
    # `clculate_local` returns list but we need rank 1 ndarray
    res = res[0].reshape(len(res[0]))
    return res

<img src="images/lab_2_gb.png" width=650>

In [11]:
print(f"0.41/0.59 = {0.41 / 0.59}")
print(f"0.59/0.41 = {0.59 / 0.41}")

0.41/0.59 = 0.6949152542372882
0.59/0.41 = 1.4390243902439024


In [12]:
# test `binary_preferences_consideration_step` with data from the lecture
res = binary_preferences_consideration_step(w_crit_test, v_test, output_P=True)
res

[[1.         1.44621063 0.99738903]
 [0.69146221 1.         0.62535662]
 [1.0026178  1.59908757 1.        ]]


array([0.36966738, 0.24740652, 0.3829261 ])

## Method for global weights calculation given arbitrary (> 2 layers) full architecture

In [13]:
def calculate_global_weights(layers, v, criteria_weights, step_method_name="distributive_synthesis_step"):
    """
    Calculates global weights of given hierarchy.
    layers: list
        numbers of elements in each layer: first layer - 1 element - global aim, ... ,
        last layer - number of alternatives
    v: list of rank 3 tensors
        local weights,matrices of pair comparisons of current layer elements w.r.t. elements of parent layer
    criteria_weights_lst: list of int
        weights of criterions on the layer[1]; total in sum give 1
    step_method_name: str
        string representation of one of the two available methods that is used iteratively.
        Available values: `distributive_synthesis_step`, `binary_preferences_consideration_step`
    return: ndarray rank 1
        global weights of alternatives
    """
    # choose method for making step from parent to children in global weight calculations
    method_step = None
    if step_method_name == "distributive_synthesis_step":
        method_step = distributive_synthesis_step
    elif step_method_name == "binary_preferences_consideration_step":
        method_step = binary_preferences_consideration_step
    else:
        raise ValueError("Wrong step method name!")
    # after each iteration we 'update' weights of our criterions for next iteration
    weights = criteria_weights
    for idx_layer in range(len(layers[1:])):
        weights = method_step(weights, v[idx_layer])
    return weights

Обрахуємо ваги рекурсивно методом дистрибутивного синтезу:

In [14]:
calculate_global_weights(layers, v, criteria_weights)

array([0.53505875, 0.26072918, 0.20421206])

Обрахуємо ваги рекурсивно методом ГВБВПА:

In [15]:
calculate_global_weights(layers, v, criteria_weights,
                         step_method_name="binary_preferences_consideration_step")

array([0.54611554, 0.24083052, 0.21305394])

# Дослідження реверсу рангів

***Реверс рангів*** – це зміна рангів альтернатив при їх оцінюванні за багатьма критеріями при додаванні/вилученні альтернативи. Множина критеріїв, ваги критеріїв і оцінки «старих» альтернатив за критеріями не змінюються.

## TEST 1: Вилучення з розгляду найменш вагомого критерія

In [16]:
# find minimum value of criterion weight
idx_min_crit_weight = np.argmin(criteria_weights)
idx_min_crit_weight

2

In [17]:
criteria_weights

array([0.28010459, 0.26678609, 0.17825941, 0.27484991])

In [18]:
criteria_weights_test_1 = list(criteria_weights[:idx_min_crit_weight]) + list(criteria_weights[(idx_min_crit_weight + 1):])
criteria_weights_test_1 = np.array(criteria_weights_test_1)
criteria_weights_test_1

array([0.28010459, 0.26678609, 0.27484991])

Модифікуємо ієрархію(зараз працюємо з (1, 3, 3, 3) а не (1, 4, 3, 3) як раніше) та видалимо надлишкові дані:

In [19]:
layers_test_1 = layers.copy()
layers_test_1[1] -= 1
layers_test_1

[1, 3, 3, 3]

In [20]:
D_test_1 = D.copy()
# from first table remove <idx_min_crit_weight> row and column
D_test_1[0] = np.delete(D_test_1[0].squeeze(), (idx_min_crit_weight), axis=0)
D_test_1[0] = np.delete(D_test_1[0], (idx_min_crit_weight), axis=1)
D_test_1[0] = D_test_1[0].reshape((layers_test_1[1], layers_test_1[1], layers_test_1[0]))
# from second table remove slice corresponding to the <idx_min_crit_weights>
D_test_1[1] = np.delete(D_test_1[1], (idx_min_crit_weight), axis=2)

In [21]:
print_D(D_test_1)

(3, 3, 1)
[[[ 1.  ]
  [ 0.25]
  [ 5.  ]]

 [[ 4.  ]
  [ 1.  ]
  [20.  ]]

 [[ 0.2 ]
  [ 0.05]
  [ 1.  ]]]
********************************************************************************
(3, 3, 3)
[[[1.00000000e+00 1.00000000e+00 1.00000000e+00]
  [1.25000000e-01 2.00000000e+00 3.33333333e-01]
  [8.00000000e+00 2.00000000e-01 7.00000000e+00]]

 [[8.00000000e+00 5.00000000e-01 3.00000000e+00]
  [1.00000000e+00 1.00000000e+00 1.00000000e+00]
  [6.40000000e+01 1.00000000e-01 2.10000000e+01]]

 [[1.25000000e-01 5.00000000e+00 1.42857143e-01]
  [1.56250000e-02 1.00000000e+01 4.76190476e-02]
  [1.00000000e+00 1.00000000e+00 1.00000000e+00]]]
********************************************************************************
(3, 3, 3)
[[[1.00000000e+00 1.00000000e+00 1.00000000e+00]
  [1.11111111e-01 3.00000000e+00 6.00000000e+00]
  [5.00000000e+00 1.00000000e+00 4.00000000e+00]]

 [[9.00000000e+00 3.33333333e-01 1.66666667e-01]
  [1.00000000e+00 1.00000000e+00 1.00000000e+00]
  [4.50000000e+01 

In [22]:
v_test_1 = calculate_local(D_test_1)
v_test_1

[array([[0.19230769],
        [0.76923077],
        [0.03846154]]), array([[0.10958904, 0.15384615, 0.24137931],
        [0.87671233, 0.07692308, 0.72413793],
        [0.01369863, 0.76923077, 0.03448276]]), array([[0.09803922, 0.42857143, 0.70588235],
        [0.88235294, 0.14285714, 0.11764706],
        [0.01960784, 0.42857143, 0.17647059]])]

Обрахуємо ваги рекурсивно методом дистрибутивного синтезу:

In [23]:
calculate_global_weights(layers_test_1, v_test_1, criteria_weights_test_1)

array([0.54460831, 0.23780446, 0.21758723])

Обрахуємо ваги рекурсивно методом ГВБВПА:

In [24]:
calculate_global_weights(layers_test_1, v_test_1, criteria_weights_test_1,
                        step_method_name="binary_preferences_consideration_step")

array([0.52809403, 0.2611397 , 0.21076626])

## Висновок TEST 1

Після вилучення з розгляду найменш вагомого критерія ***реверсу рангів не відбулося*** в жодному з розглянутих методів синтезу: дистрибутивний, ГВБВПА

## TEST 2: дублювання довільної альтернативи

In [25]:
# random alternative to copy and add to hierarchy
idx_to_copy = np.random.randint(layers[1]) - 1
idx_to_copy

2

In [26]:
# number of times we add copy of chosen alternative to hierarchy
num_of_copies = np.random.randint(3) + 1
num_of_copies

2

In [27]:
criteria_weights_test_2 = criteria_weights.copy()
criteria_weights_test_2

array([0.28010459, 0.26678609, 0.17825941, 0.27484991])

In [28]:
layers_test_2 = layers.copy()
layers_test_2[-1] += num_of_copies
layers_test_2

[1, 4, 3, 5]

Тобто ми працюватимемо з альтернативами $a_1, a_2, a_3, a_3, a_3$ на останньому рівні.
Отже матриці парних порівнянь мусять бути наступних розмірностей:
* (4, 4, 1)
* (3, 3, 4)
* (5, 5, 3)

Розмірність останньої матриці більша, проте нам не потрібно генерувати ніяких додаткових значень.

In [29]:
# modify matrix of pair comparisons
D_test_2 = D.copy()
# we add all new copies to the end of the alternatives list
old_shape = D_test_2[2].shape
new_shape = (old_shape[0] + num_of_copies, old_shape[1] + num_of_copies, old_shape[2])
tmp = np.ones(new_shape)
# copy old array to temporary one
tmp[0:old_shape[0], 0:old_shape[1], 0:old_shape[2]] = D_test_2[2][:, :, :]
# duplicate comparison matrices in temporary array
for copy_incr in range(num_of_copies):
    tmp[:old_shape[0], old_shape[1] + copy_incr, :old_shape[2]] = D_test_2[2][:, idx_to_copy, :]
    tmp[old_shape[0] + copy_incr, :old_shape[1], :old_shape[2]] = D_test_2[2][idx_to_copy, :, :]
D_test_2[2] = tmp

In [30]:
print_D(D_test_2)

(4, 4, 1)
[[[1.00e+00]
  [2.50e-01]
  [1.25e-01]
  [5.00e+00]]

 [[4.00e+00]
  [1.00e+00]
  [5.00e-01]
  [2.00e+01]]

 [[8.00e+00]
  [2.00e+00]
  [1.00e+00]
  [4.00e+01]]

 [[2.00e-01]
  [5.00e-02]
  [2.50e-02]
  [1.00e+00]]]
********************************************************************************
(3, 3, 4)
[[[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
  [1.25000000e-01 2.00000000e+00 1.00000000e+00 3.33333333e-01]
  [8.00000000e+00 2.00000000e-01 3.33333333e-01 7.00000000e+00]]

 [[8.00000000e+00 5.00000000e-01 1.00000000e+00 3.00000000e+00]
  [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
  [6.40000000e+01 1.00000000e-01 3.33333333e-01 2.10000000e+01]]

 [[1.25000000e-01 5.00000000e+00 3.00000000e+00 1.42857143e-01]
  [1.56250000e-02 1.00000000e+01 3.00000000e+00 4.76190476e-02]
  [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]]]
********************************************************************************
(5, 5, 3)
[[[1.00000

In [31]:
v_test_2 = calculate_local(D_test_2)
v_test_2

[array([[0.07575758],
        [0.3030303 ],
        [0.60606061],
        [0.01515152]]),
 array([[0.10958904, 0.15384615, 0.2       , 0.24137931],
        [0.87671233, 0.07692308, 0.2       , 0.72413793],
        [0.01369863, 0.76923077, 0.6       , 0.03448276]]),
 array([[0.09433962, 0.23076923, 0.52173913],
        [0.8490566 , 0.07692308, 0.08695652],
        [0.01886792, 0.23076923, 0.13043478],
        [0.01886792, 0.23076923, 0.13043478],
        [0.01886792, 0.23076923, 0.13043478]])]

Обрахуємо ваги рекурсивно методом дистрибутивного синтезу:

In [32]:
calculate_global_weights(layers_test_2, v_test_2, criteria_weights_test_2)

array([0.3803267 , 0.2217492 , 0.13264137, 0.13264137, 0.13264137])

Обрахуємо ваги рекурсивно методом ГВБВПА:

In [33]:
calculate_global_weights(layers_test_2, v_test_2, criteria_weights_test_2,
                        step_method_name="binary_preferences_consideration_step")

array([0.39958081, 0.15895014, 0.14715635, 0.14715635, 0.14715635])

## Висновок TEST 2

Після дублювання альтернативи ***реверсу рангів не відбулося*** в жодному з розглянутих методів синтезу: дистрибутивний, ГВБВПА.